We're going to build a simple Agent system for generating cold sales outreach emails:

1. Agent workflow

2. Use of tools to call functions

3. Agent collaboration via Tools and Handoffs

In [7]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio


In [2]:
load_dotenv(override=True)

True

In [3]:
instructions1 = "You are a news reporter working for English Premier League, \
    You write professional, serious news from 2024/2025 season."
instructions2 = "You are a humorous, engaging news reporter working for English Premier League, \
    You write witty, engaging news that are likely to get a response from 2024/2025 season."
instructions3 = "You are a busy news reporter working for English Premier League, \
    You write concise, to the point news."

In [4]:
news_agent1 = Agent(
        name="Professional News Reporter",
        instructions=instructions1,
        model="gpt-4o-mini"
)

news_agent2 = Agent(
        name="Engaging News Reporter",
        instructions=instructions2,
        model="gpt-4o-mini"
)

news_agent3 = Agent(
        name="Busy News Reporter",
        instructions=instructions3,
        model="gpt-4o-mini"
)


In [5]:
result = Runner.run_streamed(news_agent1, input="Write a news from English Premier League 2024/2025 season.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

**Premier League Update: October 2024** 

In a thrilling weekend of action across the English Premier League, Manchester City reaffirmed their status as title contenders with a decisive 4-1 victory over Arsenal at the Etihad Stadium. A hat-trick from Erling Haaland, coupled with a stunning long-range effort from Phil Foden, showcased City's attacking prowess, leaving Arsenal searching for answers.

Meanwhile, Liverpool produced a late comeback against Chelsea at Anfield, with Mohamed Salah scoring a dramatic equalizer in the dying minutes after Chelsea had taken a 2-1 lead. The match solidified Liverpool’s position among the top four, while Chelsea continues to struggle for consistent form under manager Mauricio Pochettino.

In other results, Newcastle United maintained their strong start to the season with a 3-0 win against Wolverhampton Wanderers. The Magpies have climbed to third in the table, with standout performances from summer signings and a solid defensive display spearheaded 

In [8]:
message = "Write a news from English Premier League 2024/2025 season."

with trace("Parallel news from EPL 2024/2025 season."):
    results = await asyncio.gather(
        Runner.run(news_agent1, message),
        Runner.run(news_agent2, message),
        Runner.run(news_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

**English Premier League Update: Week 10 Highlights**

*London, October 15, 2024* – The English Premier League saw thrilling action this past weekend as Week 10 delivered unforgettable moments and crucial results that could shape the title race.

**Manchester City vs. Arsenal**

In a highly anticipated clash at the Etihad Stadium, Manchester City edged out Arsenal with a narrow 2-1 victory. City’s Erling Haaland continued his prolific form, scoring both goals for the home side, bringing his tally to 12 for the season. Arsenal's late response came through Bukayo Saka, who found the net in the 78th minute, adding intrigue to the closing stages. This result keeps City firmly in contention for the title as they sit just two points behind leaders Liverpool.

**Liverpool Maintains Top Spot**

Liverpool remains unbeaten after a commanding 3-0 victory over West Ham United at Anfield. Mohamed Salah was once again pivotal for the Reds, scoring two goals and providing an assist. Manager Jürgen Kl

In [ ]:
news_picker = Agent(
    name="news_picker",
    instructions="You pick the best news from the given options. \
        Imagine you are a customer and pick the one you are most likely to read to. \
        Do not give an explanation; reply with the selected news only.\
        Do not write just the title, provide the full news",
    model="gpt-4o-mini"
)

In [14]:
type(news_picker)

agents.agent.Agent

In [15]:
message = "Write a news from English Premier League 2024/2025 season."

with trace("Selection from news reporters"):
    results = await asyncio.gather(
        Runner.run(news_agent1, message),
        Runner.run(news_agent2, message),
        Runner.run(news_agent3, message),
    )

    outputs = [result.final_output for result in results]

    news = "News reports from EPL 2024/2025 season:\n\n".join(outputs)

    best = await Runner.run(news_picker, news)

    print(f"Best news reports:\n{best.final_output}")


Best news reports:
**Early Season Surprises Shake Up Premier League Contenders**

As the 2024/2025 Premier League season enters its second month, the race for the title is shaping up to be one of the most unpredictable in recent memory. Early fixtures have not only delivered thrilling matches but have also placed several teams in unexpected positions on the table.

**Tactical Innovations and Standout Performances**

One of the standout stories of the season has been the tactical acumen displayed by newly appointed managers across multiple clubs. Liverpool, under the guidance of José Mourinho, is showcasing a blend of attacking flair and defensive solidity that has seen them remain unbeaten in their first eight matches. Their latest victory—a convincing 3-1 win against Chelsea—demonstrated Mourinho’s strategic prowess as they transitioned effectively from defense to attack.

Meanwhile, Arsenal, led by emerging talent manager Emma Hayes, has taken the league by storm, currently sitting s

Now go and check out the trace:

https://platform.openai.com/traces

### Part 2: use of tools

In [16]:
news_agent1 = Agent(
        name="Professional News Reporter",
        instructions=instructions1,
        model="gpt-4o-mini"
)

news_agent2 = Agent(
        name="Engaging News Reporter",
        instructions=instructions2,
        model="gpt-4o-mini"
)

news_agent3 = Agent(
        name="Busy News Reporter",
        instructions=instructions3,
        model="gpt-4o-mini"
)


In [17]:
news_agent1

Agent(name='Professional News Reporter', instructions='You are a news reporter working for English Premier League,     You write professional, serious news from 2024/2025 season.', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

### Steps 2 and 3: Tools and Agent interactions

In [19]:
import requests

In [18]:
telegram_bot_token = os.environ.get("TELEGRAM_BOT_TOKEN")
chat_id = os.environ.get("TELEGRAM_CHAT_ID")

In [21]:
@function_tool
def send_telegram_message(body: str):
    """ Send a message with the given body to all readers. """
    url = f"https://api.telegram.org/bot{telegram_bot_token}/sendMessage"
    payload = {"chat_id": chat_id, "text": body}
    try:
        response = requests.post(url, data=payload)
        if response.status_code == 200:
            print("Notification sent successfully!")
        else:
            print("Failed to send the message:", response.text)
    except Exception as e:
        print("Error sending Telegram message:", e)

This has automatically been converted into a tool, with the boilerplate json created

In [22]:
send_telegram_message

FunctionTool(name='send_telegram_message', description='Send a message with the given body to all readers.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_telegram_message_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019006D87F60>, strict_json_schema=True)

And you can also convert an Agent into a tool


In [23]:
tool1 = news_agent1.as_tool(tool_name="news_agent1", tool_description="Write a news from English Premier League 2024/2025 season.")
tool1

FunctionTool(name='news_agent1', description='Write a news from English Premier League 2024/2025 season.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'news_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019008DCEF20>, strict_json_schema=True)

In [25]:
description = "Write a news from English Premier League 2024/2025 season."

tool1 = news_agent1.as_tool(tool_name="news_agent1", tool_description="Write a news from English Premier League 2024/2025 season.")
tool2 = news_agent2.as_tool(tool_name="news_agent2", tool_description="Write a news from English Premier League 2024/2025 season.")
tool3 = news_agent3.as_tool(tool_name="news_agent3", tool_description="Write a news from English Premier League 2024/2025 season.")

tools = [tool1, tool2, tool3, send_telegram_message]
tools

[FunctionTool(name='news_agent1', description='Write a news from English Premier League 2024/2025 season.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'news_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019008DCF240>, strict_json_schema=True),
 FunctionTool(name='news_agent2', description='Write a news from English Premier League 2024/2025 season.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'news_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019008DCC9A0>, strict_json_schema=True),
 FunctionTool(name='news_agent3', description='Write a news from English Premier League 2024/2025 season.', params_json_sche

### And now it's time for our News Manager - our planning agent

In [26]:
instructions = "You are a news manager for English Premier League. You use the tools given to you to generate news from English Premier League season 2024/2025. \
    You never generate news yourself; you always use the tools. \
    You try all 3 news_agent tools once before choosing the best one. \
    You pick the single best news and use the send_telegram_message tool to send the best message (and only the best message) to the user."

news_manager = Agent(name="News manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Write a news from English Premier League 2024/2025 season to 'Mert Bayraktar'"

with trace("News manager"):
    result = await Runner.run(news_manager, message)

Notification sent successfully!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across

In [32]:
epl_instructions = "You will criticise the given news according to a Liverpool fan."

criticism_instructions = "You will criticise the given news according to a Manchester United fan."

epl_writer = Agent(name="EPL Writer", instructions=epl_instructions, model="gpt-4o-mini")
epl_tool = epl_writer.as_tool(tool_name="epl_writer", tool_description="Criticise the news from a Liverpool fan perspective")

criticism_agent = Agent(name="Criticism Agent", instructions=criticism_instructions, model="gpt-4o-mini")
criticism_tool = criticism_agent.as_tool(tool_name="criticism_agent",tool_description="Criticise the news from a Manchester United fan perspective")


In [33]:
@function_tool
def send_telegram_message(body: str):
    """ Send a message with the given body to all readers. """
    url = f"https://api.telegram.org/bot{telegram_bot_token}/sendMessage"
    payload = {"chat_id": chat_id, "text": body}
    try:
        response = requests.post(url, data=payload)
        if response.status_code == 200:
            print("Notification sent successfully!")
        else:
            print("Failed to send the message:", response.text)
    except Exception as e:
        print("Error sending Telegram message:", e)

In [34]:
tools = [epl_tool, criticism_tool, send_telegram_message]
tools

[FunctionTool(name='epl_writer', description='Criticise the news from a Liverpool fan perspective', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'epl_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019007A674C0>, strict_json_schema=True),
 FunctionTool(name='criticism_agent', description='Criticise the news from a Manchester United fan perspective', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'criticism_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019007A64220>, strict_json_schema=True),
 FunctionTool(name='send_telegram_message', description='Send a message with the given body to all readers.', params_json_sc

In [35]:
instructions = "You are a news writer and critic. You receive a message to write a news on. \
You first use the epl_writer tool to criticize it, then use the criticism _tool to criticize it. \
Finally, you use the send_telegram_message tool to send the message to the reader."

telegram_agent = Agent(
    name="telegram agent",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Criticize the given news."
)

In [36]:
tools = [tool1, tool2, tool3]
handoffs = [telegram_agent]
print(tools)
print(handoffs)

[FunctionTool(name='news_agent1', description='Write a news from English Premier League 2024/2025 season.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'news_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019008DCF240>, strict_json_schema=True), FunctionTool(name='news_agent2', description='Write a news from English Premier League 2024/2025 season.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'news_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019008DCC9A0>, strict_json_schema=True), FunctionTool(name='news_agent3', description='Write a news from English Premier League 2024/2025 season.', params_json_schema

In [38]:
instructions = "You are a news manager for English Premier League. You use the tools given to you to generate news from English Premier League season 2024/2025. \
    You never generate news yourself; you always use the tools. \
    You try all 3 news_agent tools once before choosing the best one. \
    You can use the tools multiple times if you're not satisfied with the results from the first try. \
    You select the single best news using your own judgement of which news will be most effective. \
    After picking the news, you handoff to the telegram agent to write and criticize the news."

news_manager = Agent(name="News manager",
                    instructions=instructions,
                    tools=tools, 
                    handoffs=handoffs, 
                    model="gpt-4o-mini")

message = "Write a news from English Premier League 2024/2025 season to 'Mert Bayraktar'"

with trace("Critic"):
    result = await Runner.run(news_manager, message)

Notification sent successfully!
Notification sent successfully!
Notification sent successfully!
Notification sent successfully!
